# 4.2 模型参数的访问、初始化和共享

In [23]:
import torch
from torch import nn
from torch.nn import init

print(torch.__version__)

1.1.0


In [24]:
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()
print(net(X))
print(Y)

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
tensor([[-0.2716],
        [-0.3562]], grad_fn=<AddmmBackward>)
tensor(-0.6278, grad_fn=<SumBackward0>)


## 4.2.1 访问模型参数

In [25]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weighttorch.Size([3, 4])
0.biastorch.Size([3])
2.weighttorch.Size([1, 3])
2.biastorch.Size([1])


In [26]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weighttorch.Size([3, 4])<class 'torch.nn.parameter.Parameter'>
biastorch.Size([3])<class 'torch.nn.parameter.Parameter'>


In [27]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [28]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[-0.4492, -0.1990,  0.2349,  0.3995],
        [ 0.3093, -0.3330,  0.1057,  0.1594],
        [ 0.3454,  0.1896, -0.0823,  0.4216]])
None
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.4841, 0.1372, 0.2401, 0.4454],
        [0.3981, 0.3066, 0.2859, 0.4157]])


## 4.2.2 初始化模型参数

In [29]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weighttensor([[-0.0058,  0.0238,  0.0145, -0.0039],
        [-0.0028,  0.0043,  0.0023, -0.0008],
        [ 0.0032,  0.0017,  0.0005, -0.0075]])
2.weighttensor([[-0.0033,  0.0052, -0.0137]])


In [30]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.biastensor([0., 0., 0.])
2.biastensor([0.])


## 4.2.3 自定义初始化方法

In [31]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

In [32]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weighttensor([[-0.0000, -0.0000,  8.2532,  0.0000],
        [ 5.9841, -0.0000, -0.0000, -9.7810],
        [-8.0842, -0.0000,  8.1638, -0.0000]])
2.weighttensor([[ 8.4489, -0.0000, -8.7518]])


In [33]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.biastensor([1., 1., 1.])
2.biastensor([1.])


## 4.2.4 共享模型参数

In [34]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weighttensor([[3.]])


In [35]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [39]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[24.]])
